## Data Exploration Brazilian Fares 


In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [97]:
# Imports
import os
import pandas as pd
import requests
#from google.colab import auth
#from google.colab import drive

# Data Viz
import matplotlib.pyplot as plt
import seaborn as sns

# Downloading the cleaned data from the CSV file

In [98]:
# Path to the CSV file
my_path = "/Users/williamtordo/code/skyai/data/brazil_flight_fares_final.csv"

# Google Drive link
#https://drive.google.com/file/d/1kuQARGVH1JmeIjbB393AYTHI3fQlD696/view?usp=drive_link

#export GOOGLE_APPLICATION_CREDENTIALS="/path/to/your/service-account-key.json"


# Reading the CSV file into a DataFrame
try:
    my_df = pd.read_csv(my_path)
    print(my_df.head())  # Display the first few rows of the DataFrame
except FileNotFoundError:
    print(f"File not found at the specified path: {my_path}")
except pd.errors.ParserError:
    print("There was an error parsing the CSV file. Please check its format.")


   Year  Month Airline Departure Arrival  Price in BRL  Seats
0  2023      1     ABJ      SBSV    SIRI         650.0     17
1  2023      1     ABJ      SBSV    SIRI         850.0     23
2  2023      1     ABJ      SBSV    SIRI        1050.0      6
3  2023      1     ABJ      SBSV    SIRI        1250.0      1
4  2023      1     ABJ      SBSV    SNCL         450.0      1


In [99]:
my_df

,Year,Month,Airline,Departure,Arrival,Price in BRL,Seats
0,2023,1,ABJ,SBSV,SIRI,650.0,17
1,2023,1,ABJ,SBSV,SIRI,850.0,23
2,2023,1,ABJ,SBSV,SIRI,1050.0,6
3,2023,1,ABJ,SBSV,SIRI,1250.0,1
4,2023,1,ABJ,SBSV,SNCL,450.0,1
...,...,...,...,...,...,...,...
12268537,2021,11,TAM,SBVT,SBUL,742.0,3
12268538,2021,11,TAM,SBVT,SBUL,925.0,2
12268539,2021,11,TAM,SBVT,SBUL,1040.0,2
12268540,2021,11,TAM,SBVT,SBUL,1512.0,1


## Focus on the top 3 Airlines (>95% of total traffic)

In [100]:
filtered_df = my_df[my_df['Airline'].isin(['AZU', 'TAM', 'GLO'])]
filtered_df['Airline'].value_counts()

Airline
AZU    5678856
TAM    3493541
GLO    3051163
Name: count, dtype: int64

In [ ]:
## Group by routes

In [101]:
route_group = filtered_df.groupby(["Departure", "Arrival"])['Price in BRL'].mean().reset_index()
#route_group.sort_values(by='Price in BRL', ascending=False)
route_group.value_counts()

Departure  Arrival  Price in BRL
SBAC       SBAE     482.900000      1
SBSI       SBJA     2070.450000     1
           SBFL     1312.548961     1
           SBFN     923.900000      1
           SBFZ     1218.405891     1
                                   ..
SBIZ       SBMA     2062.780000     1
           SBME     755.566667      1
           SBMG     1031.446380     1
           SBMK     1134.519660     1
SWYN       SBSN     2434.900000     1
Name: count, Length: 12870, dtype: int64

In [102]:
departure_airports = filtered_df['Departure'].value_counts()
#sns.histplot(filtered_df['Departure'])

In [103]:
# Grouping by routes (Departure and Arrival)
route_summary = filtered_df.groupby(["Departure", "Arrival"]).agg(
    total_seats=("Seats", "sum"),
    median_price=("Price in BRL", "median")
).reset_index()
route_summary = route_summary[route_summary['total_seats'] > 3000]

In [104]:
# Filter out routes where total_seats < 3000
Threshold = 3000
filtered_routes = route_summary[route_summary['total_seats'] > Threshold][["Departure", "Arrival"]]

# Merge the original DataFrame with the filtered routes to keep only matching pairs
cleaned_df = pd.merge(
    filtered_df,
    filtered_routes,
    on=["Departure", "Arrival"],
    how="inner"  # Ensures only matching routes are retained
)

In [105]:
route_summary.sort_values(by='median_price',ascending=False)

,Departure,Arrival,total_seats,median_price
3643,SBGR,SBCJ,3221,1422.900
8163,SBRJ,SBFN,11102,1385.890
1707,SBCJ,SBGR,3005,1382.900
2808,SBEG,SWEI,3541,1367.900
1692,SBCJ,SBCF,16248,1364.900
...,...,...,...,...
11397,SNJD,SBRF,3499,301.900
3708,SBGR,SBSR,33688,301.875
8095,SBRF,SNJD,3459,296.900
6669,SBPA,SBSM,6866,277.900


In [106]:
cleaned_df
cleaned_df["Date"] = pd.to_datetime(cleaned_df[["Year", "Month"]].assign(Day=1))


In [107]:
cleaned_df.drop(columns='Year',inplace=True)
cleaned_df.drop(columns='Month',inplace=True)

In [ ]:
cleaned_df
cleaned_df.info() # check the Date is a datetime type
cleaned_df.sort_values(by='Date',ascending=False, inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10768605 entries, 0 to 10768604
Data columns (total 6 columns):
 #   Column        Dtype         
---  ------        -----         
 0   Airline       object        
 1   Departure     object        
 2   Arrival       object        
 3   Price in BRL  float64       
 4   Seats         int64         
 5   Date          datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1), object(3)
memory usage: 492.9+ MB


# We have > 1'500 different routes considered, split across > 10mio flights recorded 

## Baseline model: we compute the median for each route and store it in our y_pred column
1 - Train-Test Split 80-20% 

In [ ]:
# Ensure the DataFrame is sorted by date
cleaned_df = cleaned_df.sort_values(by="Date").reset_index(drop=True)

# Define the split point based on the most recent 12 months
# Extract the unique months and count the total months in the dataset
unique_dates = cleaned_df["Date"].dt.to_period("M").unique()

# Identify the cutoff point for the last 6 months
test_start_date = unique_dates[-12].start_time

# Split the data into training and testing sets
df_train = cleaned_df[cleaned_df["Date"] < test_start_date]
df_test = cleaned_df[cleaned_df["Date"] >= test_start_date]

# Print shapes to verify
print(f"Training data shape: {df_train.shape}")
print(f"Testing data shape: {df_test.shape}")

# Calculate median price for each route in the Train Data Set
median_baseline = df_train.groupby(["Departure", "Arrival"]).agg(
    median_price=("Price in BRL", "median"),
    total_seats=("Seats", "sum")
).reset_index()

#median_baseline
#unique_dates

cleaned_df

Training data shape: (588568, 7)
Testing data shape: (1565153, 7)


,Airline,Departure,Arrival,Price in BRL,Seats,Date,median_price
0,TAM,SBBR,SBGL,584.90,5,2021-01-01,520.900
1,GLO,SBSP,SBMO,299.00,21,2021-01-01,892.675
2,GLO,SBSP,SBMO,384.00,6,2021-01-01,892.675
3,GLO,SBSP,SBMO,435.14,1,2021-01-01,892.675
4,GLO,SBSP,SBMO,402.90,11,2021-01-01,892.675
...,...,...,...,...,...,...,...
2153716,AZU,SBNF,SBCF,1283.90,1,2021-08-01,543.890
2153717,AZU,SBNF,SBCF,632.89,1,2021-08-01,543.890
2153718,AZU,SBNF,SBCF,618.90,2,2021-08-01,543.890
2153719,AZU,SBNF,SBCF,1073.90,1,2021-08-01,543.890


In [ ]:
# Merge median baseline with df_test
df_test = df_test.merge(median_baseline[["Departure", "Arrival", "median_price"]],
              on=["Departure", "Arrival"], how="left")

print(df_test.head())

  Airline Departure Arrival  Price in BRL  Seats       Date  median_price
0     AZU      SBNF    SBCF       1027.90      1 2021-08-01        543.89
1     AZU      SBNF    SBBR        906.89      2 2021-08-01        599.90
2     AZU      SBNF    SBBR        906.90      3 2021-08-01        599.90
3     AZU      SBNF    SBBR        938.89      2 2021-08-01        599.90
4     AZU      SBNF    SBBR        938.90      2 2021-08-01        599.90


In [118]:
# Baseline model evaluation(with MAE)
from sklearn.metrics import mean_absolute_error

# Evaluate baseline
mae_baseline = mean_absolute_error(df_test["Price in BRL"], df_test["median_price"])
print(f"Baseline MAE: {mae_baseline:.2f}")

Baseline MAE: 369.11
